In [20]:
'''从零开始实现'''
#dropout函数将以drop_prob的概率，丢弃NDArray输入X中的元素
import d2lzh as d2l
from mxnet import autograd,gluon,init,nd
from mxnet.gluon import loss as gloss,nn

def dropout(X,drop_prob):
    #Python assert（断言）用于判断一个表达式，在表达式条件为 false 的时候触发异常
    assert 0 <= drop_prob <=1
    keep_prob = 1 - drop_prob
    #这种情况下把全部元素丢弃
    if keep_prob == 0:
        return X.zeros_like()
    #nd.random.uniform 从一个均匀分布[low,high)中随机采样，注意定义域是左闭右开，即包含low，不包含high.
    #判断结果是如果小于keep_prob就返回1，属于keep_prob的概率，反之则返回0，是要丢弃的内容
    mask = nd.random.uniform(0,1,X.shape) < keep_prob 
    #最后除以keep_prob，把值放大，这样可以保持均值不变
    return mask * X /keep_prob

#如果一点都不丢弃
X = nd.arange(16).reshape(2,8)
dropout(X,0)

#如果以0.5的概率丢弃
dropout(X,0.5)

#如果全部丢弃
dropout(X,1) #这里可以看出来zero_like是把数组元素清0



[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]
<NDArray 2x8 @cpu(0)>

In [21]:
'''定义模型参数'''
#实验中，我们依旧使用Fashion-MNIST数据集 
#我们将定义一个包含两个隐藏层的多层感知机，其中两个隐藏层的输出个数是256
num_inputs,num_outputs,num_hiddens1,num_hiddens2 = 784,10,256,256
'''隐藏层1的参数'''
W1= nd.random.normal(scale=0.01,shape=(num_inputs,num_hiddens1))
#b1初始化为0了
b1 = nd.zeros(num_hiddens1)
'''隐藏层2的参数'''
W2 = nd.random.normal(scale=0.01,shape=(num_hiddens1,num_hiddens2))
b2 = nd.zeros(num_hiddens2)
'''最终的输出'''
W3 = nd.random.normal(scale=0.01,shape=(num_hiddens2,num_outputs))
b3 = nd.zeros(num_outputs)

#把参数汇总
params = [w1,b1,w2,b2,w3,b3]
for param in params:
    param.attach_grad()


In [22]:
'''定义模型'''
#(1)将全连接层和激活函数ReLU串联起来，并对每个激活函数的输出使用丢弃法
#(2)我们可以设置各个层的丢弃概率，通常的建议是把靠近输入层的丢弃概率设的小一点,我们这里第一层是0.2，第二层是0.5
#(3)我们可以通过is_training函数来判断运行模式是训练还是测试，并只在训练模式使用丢弃法
drop_prob1,drop_prob2 = 0.2,0.5

def net(X):
    #如果满足这么多列，就全部显示，不满足就少显示一行
    X = X.reshape((-1,num_inputs))
    H1 = (nd.dot(X,W1)+b1).relu()
    if autograd.is_training():#只在训练模型的时候使用丢弃法
        H1 = dropout(H1,drop_prob1) #在第一层全连接之后添加丢弃层
    H2 = (nd.dot(H1,W2)+b2).relu()
    #在训练模型时使用丢弃法
    if autograd.is_training():
        H2 = dropout(H2,drop_prob2)#在第二层全连接之后添加丢弃层
    #最终返回输出层
    return nd.dot(H2,W3) + b3


In [23]:
'''训练和测试模型'''
#这部分与之前多层感知机的训练和测试类似,batch_size相当于一批批的
num_epochs,lr,batch_size = 5,0.5,256
#交叉熵损失函数SoftmaxCrossEntropyLoss()
loss = gloss.SoftmaxCrossEntropyLoss()
#测试数据和训练数据，我们用的是封装之后的
train_iter,test_iter = d2l.load_data_fashion_mnist(batch_size)
#最后开始训练
d2l.train_ch3(net,train_iter,test_iter,loss,num_epochs,batch_size,params,lr)

epoch 1, loss 2.3022, train acc 0.104, test acc 0.100
epoch 2, loss 2.3017, train acc 0.108, test acc 0.100
epoch 3, loss 2.3014, train acc 0.106, test acc 0.100
epoch 4, loss 2.3013, train acc 0.109, test acc 0.100
epoch 5, loss 2.3012, train acc 0.111, test acc 0.100
